In [1]:
import pandas as pd
from datetime import datetime
import os

### 재난문자 location_id > 거리두기7구역으로 나누기

In [4]:
msg_location=pd.read_csv('../행정안전부_재난문자 발령현황 API 지역코드_20200331.csv',engine ='python', encoding='EUCKR')

In [5]:
msg_location.head()

,LOCATION_ID,발송지역(시도),발송지역(시군구),법정동코드,법정동(시도),법정동(시군구)
0,2,강원도,전체,4200000000,강원도,NaN
1,3,강원도,강릉시,4215000000,강원도,강릉시
2,4,강원도,고성군,4282000000,강원도,고성군
3,5,강원도,동해시,4217000000,강원도,동해시
4,6,강원도,삼척시,4223000000,강원도,삼척시


In [6]:
msg_location.columns

Index(['LOCATION_ID', '발송지역(시도)', '발송지역(시군구)', '법정동코드', '법정동(시도)', '법정동(시군구)'], dtype='object')

In [7]:
msg_location=msg_location.drop(['발송지역(시군구)','법정동코드','법정동(시도)','법정동(시군구)'],axis=1)
msg_location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   LOCATION_ID  253 non-null    int64 
 1   발송지역(시도)     253 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.1+ KB


In [8]:
msg_location.head()

,LOCATION_ID,발송지역(시도)
0,2,강원도
1,3,강원도
2,4,강원도
3,5,강원도
4,6,강원도


In [9]:
msg_location['Area']=msg_location['발송지역(시도)']
msg_location.head()

,LOCATION_ID,발송지역(시도),Area
0,2,강원도,강원도
1,3,강원도,강원도
2,4,강원도,강원도
3,5,강원도,강원도
4,6,강원도,강원도


In [10]:
msg_location['Area'].unique()

array(['강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시',
       '서울특별시', '울산광역시', '인천광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도',
       '충청북도', '세종특별자치시', '임진강 수계지역(경기도 연천군,파주시)'], dtype=object)

In [11]:
msg_location['Area']=msg_location['Area'].replace(['서울특별시','경기도','임진강 수계지역(경기도 연천군,파주시)','인천광역시'],'수도권')
msg_location['Area']=msg_location['Area'].replace(['대전광역시','세종특별자치시','충청북도','충청남도'],'충청권')
msg_location['Area']=msg_location['Area'].replace(['광주광역시','전라북도','전라남도'],'호남권')
msg_location['Area']=msg_location['Area'].replace(['대구광역시','경상북도'],'경북권')
msg_location['Area']=msg_location['Area'].replace(['부산광역시','울산광역시','경상남도'],'경남권')
msg_location['Area']=msg_location['Area'].replace(['강원도'],'강원권')
msg_location['Area']=msg_location['Area'].replace(['제주특별자치도'],'제주')

In [12]:
msg_location.isnull().sum()

LOCATION_ID    0
발송지역(시도)       0
Area           0
dtype: int64

In [13]:
msg_location

,LOCATION_ID,발송지역(시도),Area
0,2,강원도,강원권
1,3,강원도,강원권
2,4,강원도,강원권
3,5,강원도,강원권
4,6,강원도,강원권
...,...,...,...
248,6406,충청북도,충청권
249,6474,세종특별자치시,충청권
250,6474,세종특별자치시,충청권
251,6487,"임진강 수계지역(경기도 연천군,파주시)",수도권


In [14]:
msg_location.columns=['location_id','location_sido','Area']
msg_location['location_id']=msg_location['location_id'].astype(str)
msg_location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   location_id    253 non-null    object
 1   location_sido  253 non-null    object
 2   Area           253 non-null    object
dtypes: object(3)
memory usage: 6.1+ KB


In [15]:
msg_location.to_csv("msg_location.csv", index=False)

### total_msg와 합치기

In [18]:
msg_total=pd.read_csv('../safety_msg_crawling_total.csv',engine='python')

In [19]:
msg_total['create_date']=pd.to_datetime(msg_total['create_date'])

In [20]:
msg_total=msg_total[msg_total['create_date']>='2020-01-23 15:08:34']
msg_total.head()

,create_date,location_id,location_name,md101_sn,msg,send_platform
0,2021-07-02 10:24:18,104,대구광역시 전체,110649,[대구광역시] 2일 0시 기준 확진자 9명(동구3 북구2 서구1 수성구1 달서구1 ...,cbs
1,2021-07-02 10:22:59,98,광주광역시 전체,110648,[광주광역시] 6.18.(금)~6.30.(수) 보보스(BOBOS)호프(북구 대자로 ...,cbs
2,2021-07-02 10:16:02,23,경기도 고양시,110647,"[고양시청]7.1.(목) 일일 확진자 33명(덕양구9,일산동구14,일산서구9,타지역...",cbs
3,2021-07-02 10:11:29,58,경상남도 남해군,110646,[남해군청]▶7.2.(금) 09:00기준 확진자 2명 발생 ( 자가격리 중 해제 전...,cbs
4,2021-07-02 10:09:23,26,경기도 광주시,110645,[광주시청]제2차 광주시 재난기본소득(1인당 10만원) 미신청 하신 분은 7월말까지...,cbs


In [21]:
msg_location.set_index('location_id',inplace=True)
msg_total.set_index('location_id',inplace=True)

In [22]:
msg_total_id=msg_total.join(msg_location)
msg_total_id

,create_date,location_name,md101_sn,msg,send_platform,location_sido,Area
location_id,,,,,,,
10,2021-06-27 10:23:41,강원도 영월군,110127,"[영월군청] 6/27(일) 확진자 영월읍 1명 발생(영월43번), 감염경로 역학조사...",cbs,강원도,강원권
10,2021-06-24 19:47:20,강원도 영월군,109933,[행정안전부] 오늘 19시45분 강원(영월) 호우주의보 발효. 대중교통을 이용하시고...,cbs,강원도,강원권
10,2021-05-24 09:34:44,강원도 영월군,106650,"[영월군청]5.24.(월) 1명 확진(영월 41번), 역학조사 중이며, 이동 동선 ...",cbs,강원도,강원권
10,2021-05-22 12:13:45,강원도 영월군,106466,"[영월군청]5.22.(토) 1명 확진(영월 40번), 접촉자 파악 및 방역소독 완료...",cbs,강원도,강원권
10,2021-05-04 13:11:21,강원도 영월군,104167,"[영월군청]5.4.(화, 13시) 1명 확진(영월39번) 자가격리중으로 지역 내 이...",cbs,강원도,강원권
...,...,...,...,...,...,...,...
"99,100,103,185,188,194,196,199",2020-07-13 08:02:46,"광주광역시 광산구,광주광역시 남구,광주광역시 서구,전라남도 나주시,전라남도 무안군,...",40493,"[영산강홍수통제소]오늘 08:00 황룡강 광주광역시(장록교) 홍수주의보 발령, 안전...",cbs,NaN,NaN
"99,100,185,188,194,199",2020-08-07 19:40:54,"광주광역시 광산구,광주광역시 남구,전라남도 나주시,전라남도 무안군,전라남도 영암군,...",42841,"[영산강홍수통제소]오늘 19:40 영산강 나주시(나주대교) 홍수경보 변경발령, 홍수...",cbs,NaN,NaN
"99,100,185,188,194,199",2020-08-07 16:39:21,"광주광역시 광산구,광주광역시 남구,전라남도 나주시,전라남도 무안군,전라남도 영암군,...",42757,"[영산강홍수통제소]오늘 16:40 영산강 나주시(나주대교) 홍수주의보 발령, 안전에...",cbs,NaN,NaN


In [23]:
msg_total_id=msg_total_id.reset_index(drop=False)
msg_total_id.head()

,location_id,create_date,location_name,md101_sn,msg,send_platform,location_sido,Area
0,10,2021-06-27 10:23:41,강원도 영월군,110127,"[영월군청] 6/27(일) 확진자 영월읍 1명 발생(영월43번), 감염경로 역학조사...",cbs,강원도,강원권
1,10,2021-06-24 19:47:20,강원도 영월군,109933,[행정안전부] 오늘 19시45분 강원(영월) 호우주의보 발효. 대중교통을 이용하시고...,cbs,강원도,강원권
2,10,2021-05-24 09:34:44,강원도 영월군,106650,"[영월군청]5.24.(월) 1명 확진(영월 41번), 역학조사 중이며, 이동 동선 ...",cbs,강원도,강원권
3,10,2021-05-22 12:13:45,강원도 영월군,106466,"[영월군청]5.22.(토) 1명 확진(영월 40번), 접촉자 파악 및 방역소독 완료...",cbs,강원도,강원권
4,10,2021-05-04 13:11:21,강원도 영월군,104167,"[영월군청]5.4.(화, 13시) 1명 확진(영월39번) 자가격리중으로 지역 내 이...",cbs,강원도,강원권


In [24]:
msg_total_id['Area'].isnull().sum()

1099

In [25]:
msg_total_id.to_csv("msg_total_id.csv", index=False)